# Bosch Kaggle Script
## To Do:
* Objectify
* LabelEncode Data?
* Decision Tree
* K-cross validation (going to be kind of tricky) Could just do within batches


In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import subprocess
import time
import sys
import xgboost as xgb
from collections import defaultdict

In [2]:
# Globals

# Chunksize that seems to work for 32GB of RAM
# Possible modify this
# chunksize = 50000
chunksize = 5000

file_names = {}
file_names['train'] = {}
file_names['test'] = {}

# file_names['train']['date'] = 'data/train_date.csv'
file_names['train']['categorical'] = 'data/train_categorical.csv'
file_names['train']['numeric'] = 'data/train_numeric.csv'
# file_names['test']['date'] = 'data/test_date.csv'
file_names['test']['categorical'] = 'data/test_categorical.csv'
file_names['test']['numeric'] = 'data/test_numeric.csv'

model_dir = 'data/models/'
sub_dir = 'data/submissions/'

# This is for the categories and date
d = defaultdict(LabelEncoder)


# XGBoost stuff
num_rounds = 2
params = {}


In [3]:
def train_and_save_models():
    # This is what will be returned
#     nlines = subprocess.check_output('wc -l %s' % file_names['train']['numeric'], shell=True)
#     nlines = int(nlines.split()[0])
    nlines = 100000
    train_types = {}
    train_columns = {}
    
    for file_name in file_names['train']:
        train_data = pd.read_csv(file_names['train'][file_name], nrows=2)
        train_types[file_name] = train_data.dtypes
        train_columns[file_name] = train_data.columns
    
    time_start = time.time()
    total_start = time_start
    
    total_sum = {}
    
    total_sum[0] = 0
    total_sum[1] = 0
    
    for i in range(1, nlines, chunksize):
        print('training on lines: %d - %d' % (i-1, i+chunksize-1))
        train_data_parts = {}
        for file_name in file_names['train']:
            train_data_parts[file_name] = pd.read_csv(
                file_names['train'][file_name],
                index_col=0,
                header=None,
                names=train_columns[file_name],
                nrows=chunksize,
                skiprows=i,
                dtype=train_types[file_name]
                )
        
        # Let's normalize or whatever
        train_data_df = train_data_parts['numeric'].join(train_data_parts['categorical']).apply(LabelEncoder().fit_transform)
        
        d_train = xgb.DMatrix(train_data_df.as_matrix(), label=train_data_df['Response'])
        
        # run this shit
        bst = xgb.cv(params, d_train, num_rounds, nfold=5, metrics={'error'}, verbose_eval=True)
        bst.save.save_model('data/models/test_model.' + str(i) + '.model')
    
    return


In [4]:
def test_and_make_submission():    
    # all of the line numbers are the same (I hope or we have bigger issues)
#     nlines = subprocess.check_output('wc -l %s' % file_names['test']['numeric'], shell=True)
#     nlines = int(nlines.split()[0])
    nlines = 1000
    test_types = {}
    test_columns = {}
    
    for file_name in file_names['test']:
        test_data = pd.read_csv(file_names['test'][file_name], nrows=2)
        test_types[file_name] = test_data.dtypes
        test_columns[file_name] = test_data.columns
    
    for file_name in file_names['test']:
        test_data_parts[file_name] = pd.read_csv(
                file_names['test'][file_name],
                index_col=0,
                header=None,
                names=train_columns[file_name],
                nrows=chunksize,
                skiprows=i,
                dtype=test_types[file_name]
                )
        
    time_start = time.time()
    total_start = time_start
      
    
    

    
        
    
#     data.to_csv('submission.csv', mode='a', header=False)



In [5]:
train_and_save_models()
# test_and_make_submission()

training on lines: 0 - 5000


/usr/local/lib/python2.7/dist-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


[0]	train-error:0+0	test-error:0+0
[1]	train-error:0+0	test-error:0+0


AttributeError: 'DataFrame' object has no attribute 'save'

### Important Data

#### Data Exploration

Overall values counts
{0: 99432, 1: 568}

#### Benchmarks

Test size is 1m lines
This will need to be divided by 3 to handle the 3 different tables

Chunk Size | Time to read (seconds)
 --- | --- 
10k |271.936368942
25k |179.683248997
50k |163.830753088
75k | 158.054757118
100k |145.901106119
150k |150.089717865
250k |

In [ ]:
bst.best_iteration
